In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import time
import numpy as np
'''
import sys
sys.path.append('Classes')
from arpy import *
'''
import arpa
from tqdm import tqdm

In [ ]:
lm = arpa.loadf('../../round1/practiseLM.arpa')
print('Pgt(' + str(lm[0].p("ABOUT CERTAIN AREAS")) + ')')

In [ ]:
#start = time.time()
#file = '../../counts/08_14counts.txt'
file = '../../counts/2000counts.txt'
first_read = open(file ,'r')

num_lines = sum(1 for line in open(file,'r'))
#print(num_lines)

ngram_dict = {}
for x in tqdm(first_read, total=num_lines):
    line = x.split('\t')
    r = int(line[-1])
    ngram_dict[line[0]] = r

#print(ngram_dict['H. I. V.'])
#end = time.time()
#print('time: ' + str(end-start))